In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_six_month'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.1]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.5440539281833272, 0.5561574378142129)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_six_month'].values

## columns 
cols = KY_X.columns[1:]

In [9]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.0015, cols, 816)
single_stump_model['features']

['age_at_current_charge24',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_charges1',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_traffic1',
 'p_drug1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'one_year1',
 'five_year1',
 'current_violence1']

In [10]:
c_grid={'C': [0.001, 0.0015]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [11]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}, {'C': 0.0015}],
 0.5331056471740248,
 0.0006894332787947377)

### RiskSLIM

In [12]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y,  
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

16

In [13]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [14]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'misdemeanor_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/16/20 @ 01:01 PM | 213 rows in lookup table
02/16/20 @ 01:01 PM | ------------------------------------------------------------
02/16/20 @ 01:01 PM | runnning initialization procedure
02/16/20 @ 01:01 PM | ------------------------------------------------------------
02/16/20 @ 01:01 PM | CPA produced 2 cuts
02/16/20 @ 01:01 PM | running naive rounding on 4 solutions
02/16/20 @ 01:01 PM | best objective value: 0.5273
02/16/20 @ 01:01 PM | rounding produced 4 integer solutions
02/16/20 @ 01:01 PM | best objective value is 0.5145
02/16/20 @ 01:01 PM | running sequential rounding on 4 solutions
02/16/20 @ 01:01 PM | best objective value: 0.5273
02/16/20 @ 01:01 PM | sequential rounding produced 2 integer solutions
02/16/20 @ 01:01 PM | best objective value: 0.3253
02/16/20 @ 01:01 PM | polishing 6 solutions
02/16/20 @ 01:01 PM | best objective value: 0.3253
02/16/20 @ 01:01 PM | polishing produced 4 integer solutions
02/16/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/16/20 @ 01:01 PM | adding 268 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1586.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 188585 51966        cutoff              0.1569        0.1538  1080924    1.94%          rho_10 D 188585 188583     51
 192653 52613        0.1544    13        0.1569        0.1538  1099387    1.92%           rho_0 D 192653 142037     46
 196675 53012        0.1557     7        0.1569        0.1539  1116690    1.90%          rho_14 U 196675 196673     35
 200557 53569        cutoff              0.1569        0.1539  1134535    1.88%          rho_14 N 200557 200556     41
 204586 54260        0.1567    11        0.1569        0.1539  1152191    1.87%          rho_13 D 204586 204585     26
 208803 55221        0.1558    10        0.1569        0.1539  1171041    1.86%           rho_0 D 208803 208801     32
 212668 55372        0.1562    14        0.1569        0.1540  1189044    1.83%          rho_10 U 212668 212667     18
 216789 56050        cutoff              0.1569        0.1540  1207461    1.82%           rho_0 U 216789  32295     27
 220961 56860        0.1563    13        0.1569 

02/16/20 @ 01:03 PM | sequential rounding produced 2 integer solutions
02/16/20 @ 01:03 PM | best objective value: 0.3249
02/16/20 @ 01:03 PM | polishing 6 solutions
02/16/20 @ 01:03 PM | best objective value: 0.3249
02/16/20 @ 01:03 PM | polishing produced 4 integer solutions
02/16/20 @ 01:03 PM | initialization produced 9 feasible solutions
02/16/20 @ 01:03 PM | best objective value: 0.1572
02/16/20 @ 01:03 PM | ------------------------------------------------------------
02/16/20 @ 01:03 PM | completed initialization procedure
02/16/20 @ 01:03 PM | ------------------------------------------------------------
02/16/20 @ 01:03 PM | 213 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               2000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.15172795548752691
CPXPARA

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/16/20 @ 01:03 PM | adding 260 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1572.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/16/20 @ 01:38 PM | adding 276 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1576.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 176954 38011        0.1555     5        0.1560        0.1532   945513    1.77%           rho_4 D 176954 176953     50
 180544 38544        cutoff              0.1560        0.1533   958741    1.74%         alpha_7 U 180544 180543     66
 184190 39134        0.1541    14        0.1560        0.1533   973375    1.72%          rho_10 U 184190  11613     24
 187595 39126        cutoff              0.1560        0.1534   986632    1.69%        alpha_13 N 187595 187594     35
 191146 39672        cutoff              0.1560        0.1534  1001316    1.67%           rho_0 D 191146 191145     41
 194373 39555        0.1546    11        0.1560        0.1534  1015415    1.64%           rho_8 U 194373 194372     50
 198317 40392        cutoff              0.1560        0.1535  1028581    1.63%           rho_8 D 198317  31266     17
 202120 41160        0.1549    11        0.1560        0.1535  1042019    1.62%           rho_1 D 202120 202118     30
Elapsed time = 29.55 sec. (50796.40 ticks, tree 

CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               2000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.15267068614270071
CPXPARAM_MIP_Tolerances_UpperCutoff              0.15818401944051663


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/16/20 @ 01:39 PM | adding 259 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1582.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 167632 49814        0.1552     8        0.1565        0.1532  1028496    2.11%          rho_10 U 167632 167631     36
 171342 50463        0.1556     5        0.1565        0.1532  1044311    2.10%         alpha_7 D 171342 171341     48
 175020 51157        0.1554    15        0.1565        0.1532  1061878    2.08%           rho_6 U 175020 175018     38
 178989 52102        0.1543    12        0.1565        0.1533  1079040    2.07%           rho_2 U 178989 118074     33
 182770 52985        0.1564     3        0.1565        0.1533  1095806    2.05%         alpha_5 D 182770 182769     37
 186793 54076        0.1553    12        0.1565        0.1533  1111360    2.04%           rho_3 D 186793 186792     24
 190437 54860        cutoff              0.1565        0.1533  1125581    2.02%           rho_3 U 190437 121513     26
 193947 55339        cutoff              0.1565        0.1534  1142048    2.00%          rho_16 D 193947 193945     23
 197255 55580        0.1536    12        0.1565 

In [15]:
np.mean(riskslim_summary['FL_score']), np.mean(riskslim_summary['KY_validation'])

(0.5259077393391529, 0.6952463215008272)

In [16]:
#### save results
summary_misdemeanor6_ky_inter_model = {"cart": cart_summary,
                                       "ebm": ebm_summary,
                                       "stumps": stumps_summary,
                                       "riskslim": riskslim_summary}

In [17]:
path = "./results/interpretable/six-month/"
results = [["Misdemeanor",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'six-month-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)